# TP 2  Linear regression

```yaml
Date: Jan 31 2024
Name: Liyao JIAO
```

In [1]:
fn1 = "Liyao"
ln1 = "JIAO"
filename = "_".join(map(lambda s: s.strip().lower(),
["SD-TSIA204_lab2", ln1, fn1])) + ".ipynb"

In [2]:
filename

'sd-tsia204_lab2_jiao_liyao.ipynb'